In [1]:
import os
import io
import imageio
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers



In [2]:
DATASET_NAME = "hockey_fight_videos"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (36, 28, 28, 3)
NUM_CLASSES = 2
LABELS = ["no","fi"]

# OPTIMIZER
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.00001

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

## DATASET

In [3]:
import os
import random
from sklearn.model_selection import train_test_split

# veri kümenizi yükleyin
data_path = "C:/Users/Emirhan/Downloads/data"
classes = os.listdir(data_path)

videos = []
labels = []
train_files = []
val_files = []
for cls in classes:
    cls_path = os.path.join(data_path, cls)
    files = os.listdir(cls_path)
    for video in os.listdir(cls_path):
        video_path = os.path.join(cls_path, video)
        videos.append(video_path)
        label = os.path.basename(video_path).split('_')[0][0:2]
        if label not in LABELS:
            continue
        labels.append(label)



In [4]:
labels = np.array(labels)
videos = np.array(videos)

In [5]:
from sklearn.preprocessing import OneHotEncoder


# One-Hot Encoding
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(np.array(labels).reshape(-1, 1))

# Kodlanmış etiketleri kontrol etme
print(labels)


[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


c:\Users\Emirhan\anaconda3\envs\tfenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
videolar = videos
etiketler = labels

X_train, X_test, y_train1, y_test1 = train_test_split(videolar, etiketler, test_size=0.20,random_state=1)


In [7]:
import cv2
import numpy as np

# Videoların yollarını içeren bir liste oluşturma
video_paths = videos

# En kısa video uzunluğunu belirleme
min_frame_count = float('inf')

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < min_frame_count:
        min_frame_count = frame_count

# Tüm videolardan aynı sayıda frame almak için döngü
frames = []

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    selected_frames = []
    for i in range(min_frame_count):
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (28,28))
            selected_frames.append(frame)
    frames.append(np.array(selected_frames))

# Tüm videolardaki ortak frame sayısına sahip bir matris oluşturma
video_array = np.stack(frames)

# Video dizisinin şeklini yazdırma
print(video_array.shape)

(2000, 36, 28, 28, 3)


In [8]:
X_train_frame, X_test_frame, y_train, y_test = train_test_split(video_array, labels, test_size=0.20,random_state=1)

In [9]:
X_train_frame, X_val_frame, y_train, y_val = train_test_split(X_train_frame, y_train, test_size=0.15, random_state=1)

In [10]:
X_train_frame

array([[[[[226, 241, 252],
          [229, 244, 255],
          [ 78,  67,  88],
          ...,
          [  7,  12,  12],
          [155, 158, 193],
          [143, 175, 210]],

         [[226, 241, 252],
          [225, 240, 251],
          [197, 219, 237],
          ...,
          [  5,  10,  10],
          [174, 152, 171],
          [132, 160, 190]],

         [[ 51,  49,  53],
          [ 19,  23,  26],
          [  7,  15,  27],
          ...,
          [ 14,  16,  23],
          [ 96,  91,  97],
          [ 17,  12,  18]],

         ...,

         [[ 58, 130, 168],
          [ 81, 175, 223],
          [102, 192, 232],
          ...,
          [159, 158, 183],
          [ 30,  69,  96],
          [181, 223, 241]],

         [[  6,  59,  63],
          [ 28,  49,  86],
          [ 27,  64,  98],
          ...,
          [ 73,  84, 146],
          [ 60,  84, 160],
          [ 40,  55, 116]],

         [[ 35, 110, 105],
          [ 19,  54,  80],
          [ 37,  74, 108],
         

In [11]:
X_train_frame = X_train_frame/255
X_test_frame = X_test_frame/255
X_val_frame = X_val_frame/255

In [12]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(X_train_frame, y_train, "train")
validloader = prepare_dataloader(X_val_frame, y_val, "valid")
testloader = prepare_dataloader(X_test_frame, y_test, "test")

In [13]:
testloader

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 36, 28, 28, 3, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [14]:
# TUBELET EMBEDDING

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [15]:
# POSITIONAL 

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [16]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [17]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=['accuracy'
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)


    return model


model = run_experiment()

Epoch 1/100
43/43 [==============================] - 15s 94ms/step - loss: 0.7776 - accuracy: 0.5162 - val_loss: 0.7172 - val_accuracy: 0.5125
Epoch 2/100
43/43 [==============================] - 3s 65ms/step - loss: 0.6912 - accuracy: 0.5581 - val_loss: 0.6802 - val_accuracy: 0.5125
Epoch 3/100
43/43 [==============================] - 3s 63ms/step - loss: 0.6774 - accuracy: 0.5941 - val_loss: 0.6647 - val_accuracy: 0.5542
Epoch 4/100
43/43 [==============================] - 3s 63ms/step - loss: 0.6491 - accuracy: 0.6426 - val_loss: 0.6051 - val_accuracy: 0.6458
Epoch 5/100
43/43 [==============================] - 3s 63ms/step - loss: 0.5753 - accuracy: 0.6831 - val_loss: 0.5254 - val_accuracy: 0.6958
Epoch 6/100
43/43 [==============================] - 3s 63ms/step - loss: 0.5492 - accuracy: 0.6926 - val_loss: 0.5108 - val_accuracy: 0.7125
Epoch 7/100
43/43 [==============================] - 3s 64ms/step - loss: 0.5531 - accuracy: 0.7029 - val_loss: 0.5092 - val_accuracy: 0.7292
Epoch

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 36, 28, 28,  0           []                               
                                 3)]                                                              
                                                                                                  
 tubelet_embedding (TubeletEmbe  (None, 36, 128)     196736      ['input_1[0][0]']                
 dding)                                                                                           
                                                                                                  
 positional_encoder (Positional  (None, 36, 128)     4608        ['tubelet_embedding[0][0]']      
 Encoder)                                                                                     

In [19]:
import numpy as np

# test verilerinizi hazırlayın
x_test = X_test_frame

# tahminleri yapın
y_pred = model.predict(x_test)

# tahmin sonuçlarını kontrol edin
print(y_pred)  # (num_samples, num_classes)


13/13 [==============================] - 1s 24ms/step
[[3.05645231e-10 1.00000000e+00]
 [1.00000000e+00 3.84940357e-10]
 [1.00000000e+00 3.74038966e-10]
 [1.00000000e+00 1.44730394e-09]
 [1.00000000e+00 2.78655404e-10]
 [9.99921799e-01 7.82420175e-05]
 [9.99993801e-01 6.18543436e-06]
 [5.91452443e-10 1.00000000e+00]
 [1.00000000e+00 1.06880735e-08]
 [1.00000000e+00 1.73164438e-09]
 [1.00000000e+00 5.96919891e-09]
 [9.99999881e-01 1.30608072e-07]
 [1.00000000e+00 3.27991412e-10]
 [1.00000000e+00 5.67938141e-09]
 [9.38874978e-10 1.00000000e+00]
 [3.92752606e-03 9.96072531e-01]
 [1.07262776e-09 1.00000000e+00]
 [1.00000000e+00 5.02354318e-08]
 [9.99906063e-01 9.38914527e-05]
 [1.00000000e+00 7.32659977e-09]
 [1.00000000e+00 2.99314185e-10]
 [1.00000000e+00 6.05344386e-10]
 [1.82760707e-09 1.00000000e+00]
 [2.58081201e-09 1.00000000e+00]
 [1.35919365e-09 1.00000000e+00]
 [6.29547470e-10 1.00000000e+00]
 [9.99999881e-01 9.62727640e-08]
 [1.45109913e-09 1.00000000e+00]
 [8.39927849e-10 1.000

In [20]:
predictions = y_pred > 0.5

In [21]:
predictions

array([[False,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [ Tr

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       205
           1       0.80      0.81      0.80       195

   micro avg       0.81      0.81      0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400
 samples avg       0.81      0.81      0.81       400



In [23]:
y_test

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.

In [24]:
import cv2
import imageio
def video_show(single_video):
    # Video yolu
    video_path = single_video

    # Video yükleyici
    cap = cv2.VideoCapture(video_path)

    # GIF oluşturucu
    gif_frames = []

    # Video frame'lerini okuma ve GIF'e ekleme
    while True:
        # Frame okuma
        ret, frame = cap.read()

        # Eğer frame okunamadıysa döngüden çık
        if not ret:
            break

        # Frame'i BGR'den RGB'ye dönüştürme
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Frame'i GIF'e ekleme
        gif_frames.append(frame_rgb)

    # Kaynakları serbest bırakma
    cap.release()

    # GIF dosyasını oluşturma
    imageio.mimsave("video.gif", gif_frames, fps=30)

def predict(frames):
    frames = [frames] 
    frames = np.array(frames)
    y_pred = model.predict(frames)

    # tahmin sonuçlarını kontrol edin
   
    predictions = y_pred > 0.5
    print(predictions)  # (num_samples, num_classes)


In [25]:
video_show(X_train[298])
predict(X_train_frame[298])

1/1 [==============================] - 0s 105ms/step
[[False  True]]


In [26]:
X_train_frame[4][3].shape

(28, 28, 3)